<a id="1"></a><h2></h2>
<div style="background-color:rgba(0, 153, 0, 0.5);border-radius: 15px 50px;display:fill">
    <h1><center>1. Introduction</center></h1>
</div>

<div style="color:white;display:fill;border-radius:8px;
            background-color:#DAA520;font-size:150%;
            font-family:Nexa;letter-spacing:0.5px">
    <p style="padding: 8px;color:black;"><b>1.1 | Goal</b></p>
</div>

Welcome to the May edition of the 2022 Tabular Playground Series! This month's challenge is a binary classification problem that includes a number of different feature interactions. This competition is an opportunity to explore various methods for identifying and exploiting these feature interactions. It wouldn't have been possible without the open-source contributions of other participants, in particular: <br>

1) @wti200 work on feature interactions <br>
2) @cabaxiom for idenfitying unique_characters feature <br>
3) GM @ambrosm for three public notebooks which are main inspiration for this notebook <br>
4) GM @sudalairajkumar for listing all the Feature Interactions <br>




# The overview of the buidling of metaclassfier:

<div class="alert alert-block alert-info"> 
    The architecture of a meta learning model involves two or more base models, often referred to as level-0 models, and a meta-model that combines the predictions of the base models, referred to as a level-1 model.
      Level-0 Models (Base-Models): Models fit on the training data and whose predictions are compiled.<br>
      Level-1 Model (Meta-Model): Model that learns how to best combine the predictions of the base models. <br> 
      It worth noting that when the predictions from Level-0 classifiers are used as the inputs for the Level-1 classifier,extreme care care must be taken as it can lead to overfitting. The below figure represents framework of the meta learning. The Leve-0 models need to do prediction on all each single fold, and learn from every four folds from the full data.<br> 
    
</div>


<br>
<img src= "https://yanpuli.github.io/images/stacking.jpg" alt ="TPA MAY" style='width: 800px;'>

<a id="1"></a><h2></h2>
## <font color="#blue">The structure of notebook.</font>
### <a href='#1'>1. Introduction </a><br>
### <a href='#2'>2. Load data and preprocessing</a><br>
### <a href='#3'>3. Feature Engineering</a><br>
### <a href='#4'>4. Neural network based prediction</a><br>
### <a href='#5'>5. LGBM based prediction</a><br>
### <a href='#6'>6. XGB based prediction</a><br>
### <a href='#7'>7. HistGradient based prediction</a><br>
### <a href='#8'>8. Catboost based prediction</a><br>
### <a href='#9'>9. Adaboost based prediction</a><br>
### <a href='#10'>10. Random Forest based prediction</a><br>
### <a href='#11'>11. Meta classifiers- preprocessing</a><br>
### <a href='#12'>12. Meta classifiers</a><br>
### <a href='#13'>13. References</a><br>

<a id="2"></a><h2></h2>
<div style="background-color:rgba(0, 153, 0, 0.5);border-radius: 15px 50px;display:fill">
    <h1><center>2. Loading the data </center></h1>
</div>
<div style="color:white;display:fill;border-radius:8px;
            background-color:#DAA520;font-size:150%;
            font-family:Nexa;letter-spacing:0.5px">
    <p style="padding: 8px;color:black;"><b>2.1 | Train data</b></p>
</div>

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import matplotlib.style as style 
style.use('ggplot')
import warnings
warnings.filterwarnings('ignore')

import plotly.offline as py 
from plotly.offline import init_notebook_mode, iplot
py.init_notebook_mode(connected=True) # this code, allow us to work with offline plotly version
import plotly.graph_objs as go # it's like "plt" of matplot



cell_hover = {  # for row hover use <tr> instead of <td>
    'selector': 'td:hover',
    'props': [('background-color', '#ffffb3')]
}
index_names = {
    'selector': '.index_name',
    'props': 'font-style: italic; color: darkgrey; font-weight:normal;'
}
headers = {
    'selector': 'th:not(.index_name)',
    'props': 'background-color: #000000; color: white;'
}
from IPython.display import HTML
pd.set_option('display.max_columns', None)
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
import seaborn as sns
from cycler import cycler
from IPython.display import display
import datetime
import scipy.stats
import math
import random

from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.calibration import CalibrationDisplay
from sklearn.preprocessing import StandardScaler

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ReduceLROnPlateau, LearningRateScheduler, EarlyStopping
from tensorflow.keras.layers import Dense, Input, InputLayer, Add
from tensorflow.keras.utils import plot_model

plt.rcParams['axes.facecolor'] = '#0057b8' # blue
plt.rcParams['axes.prop_cycle'] = cycler(color=['#ffd700'] +
                                         plt.rcParams['axes.prop_cycle'].by_key()['color'][1:])
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
from colorama import Fore

from pandas_profiling import ProfileReport
import seaborn as sns
from sklearn import metrics
from scipy import stats
import math

from tqdm.notebook import tqdm
from copy import deepcopy
# Defining all our palette colours.
primary_blue = "#496595"
primary_blue2 = "#85a1c1"
primary_blue3 = "#3f4d63"
primary_grey = "#c6ccd8"
primary_black = "#202022"
primary_bgcolor = "#f4f0ea"

primary_green = px.colors.qualitative.Plotly[2]

plt.rcParams['axes.facecolor'] = primary_bgcolor

In [2]:
train = pd.read_csv('Data/train.csv')
test = pd.read_csv('Data/test.csv')
s=train.head()
s.style.set_table_styles([cell_hover, index_names])

,id,f_00,f_01,f_02,f_03,f_04,f_05,f_06,f_07,f_08,f_09,f_10,f_11,f_12,f_13,f_14,f_15,f_16,f_17,f_18,f_19,f_20,f_21,f_22,f_23,f_24,f_25,f_26,f_27,f_28,f_29,f_30,target
0,0,-1.373246,0.238887,-0.243376,0.567405,-0.647715,0.839326,0.113133,1,5,1,3,3,3,1,6,1,0,7,4,0.298218,-0.919717,3.058541,-2.540739,0.766952,-2.730628,-0.208177,1.363402,ABABDADBAB,67.609153,0,0,0
1,1,1.697021,-1.710322,-2.230332,-0.545661,1.113173,-1.552175,0.447825,1,3,4,0,2,3,0,1,0,4,6,0,-3.147667,-1.075434,2.179050,2.278315,-0.633658,-1.217077,-3.782194,-0.058316,ACACCADCEB,377.096415,0,0,1
2,2,1.681726,0.616746,-1.027689,0.810492,-0.609086,0.113965,-0.708660,1,0,2,6,6,4,3,1,2,2,1,4,2.820733,-3.485342,-0.784235,-1.385775,-0.520558,-0.009121,2.788536,-3.703488,AAAEABCKAD,-195.599702,0,2,1
3,3,-0.118172,-0.587835,-0.804638,2.086822,0.371005,-0.128831,-0.282575,3,2,1,0,1,6,4,2,3,3,0,3,1.081084,-2.100177,-2.343819,0.572594,-1.653213,1.686035,-2.533098,-0.608601,BDBBAACBCB,210.826205,0,0,1
4,4,1.148481,-0.176567,-0.664871,-1.101343,0.467875,0.500117,0.407515,3,3,0,4,3,0,6,0,3,3,1,0,-0.126179,0.605033,1.133665,-3.912929,-1.430366,2.127649,-3.306784,4.371371,BDBCBBCHFE,-217.211798,0,1,1


<a id="3"></a><h2></h2>
<div style="color:white;display:fill;border-radius:8px;
            background-color:#DAA520;font-size:150%;
            font-family:Nexa;letter-spacing:0.5px">
    <p style="padding: 8px;color:black;"><b>2.2 | Feature Engineering</b></p>
</div>
Este modelo abaixo é uma versão modificada dos cadernos públicos da @ambrosm. Considere votar a favor do trabalho original. <br>
Dois recursos adicionais são introduzidos (interação de recursos entre f_00, f_01 com f_26 ) <br>

In [3]:
features = [f for f in test.columns if f != 'id' and f != 'f_27']
float_features = [f for f in features if test[f].dtype == float]
for df in [train, test]:
    # Extract the 10 letters of f_27 into individual features
    for i in range(10):
        df[f'ch{i}'] = df.f_27.str.get(i).apply(ord) - ord('A')
        
    # unique_characters feature is from https://www.kaggle.com/code/cabaxiom/tps-may-22-eda-lgbm-model
    df["unique_characters"] = df.f_27.apply(lambda s: len(set(s)))
    
    # Feature interactions: create three ternary features
    # Every ternary feature can have the values -1, 0 and +1
    df['i_02_21'] = (df.f_21 + df.f_02 > 5.2).astype(int) - (df.f_21 + df.f_02 < -5.3).astype(int)
    df['i_05_22'] = (df.f_22 + df.f_05 > 5.1).astype(int) - (df.f_22 + df.f_05 < -5.4).astype(int)
    df['i_00_26'] = (df.f_00 + df.f_26 > 5.1).astype(int) - (df.f_00 + df.f_26 < -5.4).astype(int)  #addtional feature
    df['i_01_26'] = (df.f_01 + df.f_26 > 5.1).astype(int) - (df.f_01 + df.f_26 < -5.4).astype(int)  #addtional feature
    i_00_01_26 = df.f_00 + df.f_01 + df.f_26
    df['i_00_01_26'] = (i_00_01_26 > 5.0).astype(int) - (i_00_01_26 < -5.0).astype(int)
    
features = [f for f in test.columns if f != 'id' and f != 'f_27']
float_features = [f for f in features if test[f].dtype == float]
int_features = [f for f in features if test[f].dtype == int and f.startswith('f')]
ch_features = [f for f in features if f.startswith('ch')]
# test[features].head(2)
corr = train.corr().abs()
mask = np.triu(np.ones_like(corr, dtype=np.bool))

fig, ax = plt.subplots(figsize=(90, 50), facecolor=primary_bgcolor)
# ax.text(-1.1, 0.16, 'Correlation between the Continuous Features', fontsize=10, fontweight='bold', fontfamily='serif')
ax.text(-1.1, 0.3, 'There is no features that pass more than 0.32 correlation within each other', fontsize=13, fontweight='light', fontfamily='serif')


# plot heatmap
res=sns.heatmap(corr, mask=mask, annot=True, fmt=".2f", cmap='coolwarm',annot_kws={"size": 25},
            cbar_kws={"shrink": .2}, vmin=0, vmax=1)
res.set_xticklabels(res.get_xmajorticklabels(), fontsize = 28)
res.set_yticklabels(res.get_ymajorticklabels(), fontsize = 28)
# yticks
plt.yticks(rotation=0)
plt.show()


<div class="alert alert-block alert-info"> 📌 
   Ótimas notícias, pois podemos ver na figura acima que dois novos recursos que foram introduzidos neste notebook i_00_26 e i_01_26 estão mostrando uma boa correlação com o Target. Vamos usá-los  

</div>

In [4]:
# Notou-se que além de vários recursos extraídos no f_27, pela codificação do rótulo do F_27 os resultados são ligeiramente melhorados.
# existem vários rótulos exclusivos observados na coluna f_27 do conjunto de dados de teste. Portanto, os dados de treinamento e teste são mesclados para codificação de rótulo.
rom sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

temp_df =(pd.concat([train.f_27, test.f_27],ignore_index=True)).to_frame() # dataframe co

temp_df['f_27']  = le.fit_transform(temp_df['f_27'])
test['f_27']     = le.transform(test['f_27'])
train['f_27']    = le.transform(train['f_27'])

<a id="4"></a><h2></h2>
<div style="background-color:rgba(0, 153, 0, 0.5);border-radius: 15px 50px;display:fill">
    <h1><center>3. Modeling </center></h1>
</div>

<br>
<div style="color:white;display:fill;border-radius:8px;
            background-color:#DAA520;font-size:150%;
            font-family:Nexa;letter-spacing:0.5px">
    <p style="padding: 8px;color:black;"><b>3.1 | Neural network based prediction</b></p>
</div>
This below model is modified version of public notebooks of @ambrosm. Consider upvoting the original work. The main differnce is number of Epchos and features <br>

In [5]:
def my_model():
    """Simple sequential neural network with four hidden layers.
    
    Returns a (not yet compiled) instance of tensorflow.keras.models.Model.
    """
    activation = 'swish'
    inputs = Input(shape=(len(features)))
    x = Dense(64, kernel_regularizer=tf.keras.regularizers.l2(40e-6),
              activation=activation,
             )(inputs)
    x = Dense(64, kernel_regularizer=tf.keras.regularizers.l2(40e-6),
              activation=activation,
             )(x)
    x = Dense(64, kernel_regularizer=tf.keras.regularizers.l2(40e-6),
              activation=activation,
             )(x)
    x = Dense(16, kernel_regularizer=tf.keras.regularizers.l2(40e-6),
              activation=activation,
             )(x)
    x = Dense(1, #kernel_regularizer=tf.keras.regularizers.l2(1e-6),
              activation='sigmoid',
             )(x)
    model = Model(inputs, x)
    return model

plot_model(my_model(), show_layer_names=False, show_shapes=True)

('You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) ', 'for plot_model/model_to_dot to work.')


In [6]:
EPOCHS = 500
EPOCHS_COSINEDECAY = 150
CYCLES = 1
VERBOSE = 0 # set to 0 for less output, or to 2 for more output
DIAGRAMS = True
USE_PLATEAU = False
BATCH_SIZE = 2048
ONLY_FIRST_FOLD = False

# see https://keras.io/getting_started/faq/#how-can-i-obtain-reproducible-results-using-keras-during-development
np.random.seed(1)
random.seed(1)
tf.random.set_seed(1)
features = [f for f in test.columns if f != 'id']
def fit_model(X_tr, y_tr, X_va=None, y_va=None, run=0):
    """Scale the data, fit a model, plot the training history and optionally validate the model
    
    Returns a trained instance of tensorflow.keras.models.Model.
    
    As a side effect, updates y_va_pred, history_list and score_list.
    """
    global y_va_pred
    start_time = datetime.datetime.now()
    
    scaler = StandardScaler()
    X_tr = scaler.fit_transform(X_tr)
    
    if X_va is not None:
        X_va = scaler.transform(X_va)
        validation_data = (X_va, y_va)
    else:
        validation_data = None

    # Define the learning rate schedule and EarlyStopping
    lr_start=0.01
    if USE_PLATEAU and X_va is not None: # use early stopping
        epochs = EPOCHS
        lr = ReduceLROnPlateau(monitor="val_loss", factor=0.7, 
                               patience=4, verbose=VERBOSE)
        es = EarlyStopping(monitor="val_loss",
                           patience=12, 
                           verbose=1,
                           mode="min", 
                           restore_best_weights=True)
        callbacks = [lr, es, tf.keras.callbacks.TerminateOnNaN()]

    else: # use cosine learning rate decay rather than early stopping
        epochs = EPOCHS_COSINEDECAY
        lr_end = 0.0002
        def cosine_decay(epoch):
            # w decays from 1 to 0 in every cycle
            # epoch == 0                  -> w = 1 (first epoch of cycle)
            # epoch == epochs_per_cycle-1 -> w = 0 (last epoch of cycle)
            epochs_per_cycle = epochs // CYCLES
            epoch_in_cycle = epoch % epochs_per_cycle
            if epochs_per_cycle > 1:
                w = (1 + math.cos(epoch_in_cycle / (epochs_per_cycle-1) * math.pi)) / 2
            else:
                w = 1
            return w * lr_start + (1 - w) * lr_end

        lr = LearningRateScheduler(cosine_decay, verbose=0)
        callbacks = [lr, tf.keras.callbacks.TerminateOnNaN()]
        
    # Construct and compile the model
    model = my_model()
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr_start),
                  metrics='AUC',
                  loss=tf.keras.losses.BinaryCrossentropy())

    # Train the model
    history = model.fit(X_tr, y_tr, 
                        validation_data=validation_data, 
                        epochs=epochs,
                        verbose=VERBOSE,
                        batch_size=BATCH_SIZE,
                        shuffle=True,
                        callbacks=callbacks)

    history_list.append(history.history)
    callbacks, es, lr, history = None, None, None, None
    
    if X_va is None:
        print(f"Training loss: {history_list[-1]['loss'][-1]:.4f}")
    else:
        lastloss = f"Training loss: {history_list[-1]['loss'][-1]:.4f} | Val loss: {history_list[-1]['val_loss'][-1]:.4f}"
        
        # Inference for validation
        y_va_pred = model.predict(X_va, batch_size=len(X_va), verbose=0)
        #oof_list[run][val_idx] = y_va_pred
        
        # Evaluation: Execution time, loss and AUC
        score = roc_auc_score(y_va, y_va_pred)
        print(f"Fold {run}.{fold} | {str(datetime.datetime.now() - start_time)[-12:-7]}"
              f" | {lastloss} | AUC: {score:.5f}")
        score_list.append(score)
    return model, scaler


print(f"{len(features)} features")
history_list = []
score_list = []
kf = KFold(n_splits=5)
for fold, (idx_tr, idx_va) in enumerate(kf.split(train)):
    X_tr = train.iloc[idx_tr][features]
    X_va = train.iloc[idx_va][features]
    y_tr = train.iloc[idx_tr].target
    y_va = train.iloc[idx_va].target
    
    fit_model(X_tr, y_tr, X_va, y_va)
    if ONLY_FIRST_FOLD: break # we only need the first fold

print(f"OOF AUC:                       {np.mean(score_list):.5f}")

47 features


2022-05-24 10:36:52.630737: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Fold 0.0 | 04:24 | Training loss: 0.0593 | Val loss: 0.0668 | AUC: 0.99799
Fold 0.1 | 03:37 | Training loss: 0.0592 | Val loss: 0.0670 | AUC: 0.99800
Fold 0.2 | 03:39 | Training loss: 0.0591 | Val loss: 0.0676 | AUC: 0.99797
Fold 0.3 | 03:37 | Training loss: 0.0590 | Val loss: 0.0662 | AUC: 0.99803
Fold 0.4 | 03:35 | Training loss: 0.0595 | Val loss: 0.0666 | AUC: 0.99800
OOF AUC:                       0.99800


In [7]:
%%time
print(f"{len(features)} features")

X_tr = train[features]
y_tr = train.target

predtest_list_cnn = []
predtrain_list_cnn=[]
for seed in range(10):
    # see https://keras.io/getting_started/faq/#how-can-i-obtain-reproducible-results-using-keras-during-development
    print(f"{seed:2}")
    np.random.seed(seed)
    random.seed(seed)
    tf.random.set_seed(seed)
    model, scaler = fit_model(X_tr, y_tr, run=seed)
    predtest_list_cnn.append(model.predict(scaler.transform(test[features]),batch_size=BATCH_SIZE, verbose=VERBOSE))
    predtrain_list_cnn.append(model.predict(scaler.transform(train[features]),batch_size=BATCH_SIZE, verbose=VERBOSE))  

47 features
 0
Training loss: 0.0604
 1
Training loss: 0.0599
 2
Training loss: 0.0597
 3
Training loss: 0.0598
 4
Training loss: 0.0597
 5
Training loss: 0.0603
 6
Training loss: 0.0596
 7
Training loss: 0.0602
 8
Training loss: 0.0597
 9
Training loss: 0.0597
CPU times: user 51min 38s, sys: 4min 54s, total: 56min 33s
Wall time: 39min 16s


<a id="5"></a><h2></h2>
<div style="color:white;display:fill;border-radius:8px;
            background-color:#DAA520;font-size:150%;
            font-family:Nexa;letter-spacing:0.5px">
    <p style="padding: 8px;color:black;"><b>3.2 |  LGBM based prediction</b></p>
</div>

In [8]:
ntrain

900000

In [9]:
from sklearn.model_selection import KFold

features = [f for f in test.columns if f != 'id']
ntrain   = train[features].shape[0]
ntest    = test.shape[0]
SEED     = 0 # for reproducibility
NFOLDS   = 5 # set folds for out-of-fold prediction

kf = KFold(n_splits= NFOLDS, random_state=SEED, shuffle=True)

# Class to extend the Sklearn classifier
class SklearnHelper(object):
    
    def __init__(self, clf, seed=0, params=None):
        params['random_state'] = seed
        self.clf = clf(**params)

    def train(self, x_train, y_train):
        self.clf.fit(x_train, y_train)

    def predict(self, x):
        return self.clf.predict(x)
    
    def fit(self,x,y):
        return self.clf.fit(x,y)
    
    def feature_importances(self,x,y):
        print(self.clf.fit(x,y).feature_importances_)

In [11]:
def get_oof(clf, x_train, y_train, x_test):
    
    oof_train    = np.zeros((ntrain,))
    oof_test     = np.zeros((ntest,))
    oof_test_skf = np.empty((NFOLDS, ntest))

    for i, (train_index, test_index) in enumerate(kf.split(x_train)):
        
        x_tr = x_train[train_index]
        y_tr = y_train[train_index]
        x_te = x_train[test_index]

        clf.train(x_tr, y_tr)

        oof_train[test_index] = clf.predict(x_te)
        oof_test_skf[i, :] = clf.predict(x_test)

    oof_test[:] = oof_test_skf.mean(axis=0)
    
    return oof_train.reshape(-1, 1), oof_test.reshape(-1, 1)

In [12]:
%%time
from lightgbm import LGBMClassifier

lgb_params = {
    'n_estimators': 300,
    'min_child_samples': 8,
    'max_bins': 51,
    
}

# Create Numpy arrays of train, test and target ( Survived) dataframes to feed into our models
y_train = train['target'].values
x_train = train[features].values # Creates an array of the train data
x_test  = test[features].values # Creats an array of the test data

pred_test_list_clf  = []
pred_train_list_clf = []

for seed in range(1):
    
    print(f"{seed:2}")
    lgb = SklearnHelper(clf=LGBMClassifier, seed=seed, params=lgb_params)
    lgb_oof_train, lgb_oof_test = get_oof(lgb,x_train, y_train, x_test) 
    score = roc_auc_score(y_train, np.array(lgb_oof_train))
    print(score)
    pred_train_list_clf.append(lgb_oof_train)
    pred_test_list_clf.append(lgb_oof_test)
    
lgb_oof_train=np.array(pred_train_list_clf).mean(axis=0)
lgb_oof_test=np.array(pred_test_list_clf).mean(axis=0)
score = roc_auc_score(y_train, np.array(lgb_oof_train))
print("final_LGBM_AUC",score)

 0
0.9531347327843647
final_LGBM_AUC 0.9531347327843647
CPU times: total: 14min 14s
Wall time: 1min 59s


In [21]:
lgb_oof_test

array([[1.],
       [1.],
       [0.],
       ...,
       [0.],
       [0.],
       [0.]])

In [20]:
np.array(lgb_oof_train)

array([[0.],
       [1.],
       [1.],
       ...,
       [0.],
       [0.],
       [1.]])

<a id="6"></a><h2></h2>
<div style="color:white;display:fill;border-radius:8px;
            background-color:#DAA520;font-size:150%;
            font-family:Nexa;letter-spacing:0.5px">
    <p style="padding: 8px;color:black;"><b>3.3 |  XGB based prediction</b></p>
</div>

In [11]:
from xgboost import XGBClassifier
xgb_params = {
    'n_estimators': 100,
    'eval_metric': 'auc',
    'silent': 0,
    
}

# Create Numpy arrays of train, test and target ( Survived) dataframes to feed into our models
y_train = train['target'].values
x_train = train[features].values # Creates an array of the train data
x_test = test[features].values # Creats an array of the test data
pred_test_list_clf= []
pred_train_list_clf = []
for seed in range(1): # you can increase maximum value of range upto 10. I have choosen it to be 1 so that the runtime is shorter.
    print(f"{seed:2}")
    xgb = SklearnHelper(clf=XGBClassifier, seed=SEED, params=xgb_params)
    xgb_oof_train, xgb_oof_test = get_oof(xgb,x_train, y_train, x_test) 
    score = roc_auc_score(y_train, np.array(xgb_oof_train))
    print(score)
    pred_train_list_clf.append(xgb_oof_train)
    pred_test_list_clf.append(xgb_oof_test)
xgb_oof_train=np.array(pred_train_list_clf).mean(axis=0)
xgb_oof_test=np.array(pred_test_list_clf).mean(axis=0)
score = roc_auc_score(y_train, np.array(xgb_oof_train))
print("final_XGB_AUC",score)


 0
[12:21:34] WARNING: ../src/learner.cc:576: 
Parameters: { "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[12:27:36] WARNING: ../src/learner.cc:576: 
Parameters: { "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[12:33:41] WARNING: ../src/learner.cc:576: 
Parameters: { "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting

<a id="7"></a><h2></h2>
<div style="color:white;display:fill;border-radius:8px;
            background-color:#DAA520;font-size:150%;
            font-family:Nexa;letter-spacing:0.5px">
    <p style="padding: 8px;color:black;"><b>3.4 |  HistGradientBoostingClassifier</b></p>
</div>

In [12]:
%%time
from sklearn.ensemble import HistGradientBoostingClassifier
features = [f for f in test.columns if f != 'id']
Hist_params = {
    'max_iter': 200,    
}

# Create Numpy arrays of train, test and target ( Survived) dataframes to feed into our models
y_train = train['target'].values
x_train = train[features].values # Creates an array of the train data
x_test = test[features].values # Creats an array of the test data
pred_test_list_clf= []
pred_train_list_clf = []
for seed in range(1): # you can increase maximum value of range upto 10. I have choosen it to be 1 so that the runtime is shorter.
    print(f"{seed:2}")
    Hist = SklearnHelper(clf=HistGradientBoostingClassifier, seed=SEED, params=Hist_params)
    Hist_oof_train, Hist_oof_test = get_oof(Hist,x_train, y_train, x_test) 
    score = roc_auc_score(y_train, np.array(Hist_oof_train))
    print(score)
    pred_train_list_clf.append(Hist_oof_train)
    pred_test_list_clf.append(Hist_oof_test)
Hist_oof_train=np.array(pred_train_list_clf).mean(axis=0)
Hist_oof_test=np.array(pred_test_list_clf).mean(axis=0)
score = roc_auc_score(y_train, np.array(Hist_oof_train))
print("final_HIST_AUC",score)

 0
0.9461773410529398
final_HIST_AUC 0.9461773410529398
CPU times: user 9min 37s, sys: 1.93 s, total: 9min 39s
Wall time: 5min 12s


<a id="8"></a><h2></h2>
<div style="color:white;display:fill;border-radius:8px;
            background-color:#DAA520;font-size:150%;
            font-family:Nexa;letter-spacing:0.5px">
    <p style="padding: 8px;color:black;"><b>3.5 |  CatBoostClassifier</b></p>
</div>

In [13]:
%%time
from catboost import CatBoostClassifier
iterations = 4500
cat_params = {
    'iterations': iterations,
    'learning_rate': 0.02,
    'early_stopping_rounds': 150,
    'max_depth': 5,
    'eval_metric': 'Accuracy',
    'loss_function': 'Logloss',
    'verbose': int(iterations/10)    
}


# Create Numpy arrays of train, test and target ( Survived) dataframes to feed into our models
y_train = train['target'].values
x_train = train[features].values # Creates an array of the train data
x_test = test[features].values # Creats an array of the test data
pred_test_list_clf= []
pred_train_list_clf = []
for seed in range(1): # you can increase maximum value of range upto 10. I have choosen it to be 1 so that the runtime is shorter.
    print(f"{seed:2}")
    cat = SklearnHelper(clf=CatBoostClassifier, seed=SEED, params=cat_params)
    cat_oof_train, cat_oof_test = get_oof(cat,x_train, y_train, x_test) 
    score = roc_auc_score(y_train, np.array(cat_oof_train))
    print(score)
    pred_train_list_clf.append(cat_oof_train)
    pred_test_list_clf.append(cat_oof_test)
cat_oof_train=np.array(pred_train_list_clf).mean(axis=0)
cat_oof_test=np.array(pred_test_list_clf).mean(axis=0)
score = roc_auc_score(y_train, np.array(cat_oof_train))
print("final_AUC",score) 

 0
0:	learn: 0.6147194	total: 230ms	remaining: 17m 13s
450:	learn: 0.9108694	total: 1m 29s	remaining: 13m 22s
900:	learn: 0.9298986	total: 2m 58s	remaining: 11m 51s
1350:	learn: 0.9400250	total: 4m 26s	remaining: 10m 20s
1800:	learn: 0.9458542	total: 5m 54s	remaining: 8m 50s
2250:	learn: 0.9498069	total: 7m 21s	remaining: 7m 20s
2700:	learn: 0.9532153	total: 8m 47s	remaining: 5m 51s
3150:	learn: 0.9558333	total: 10m 15s	remaining: 4m 23s
3600:	learn: 0.9579236	total: 11m 41s	remaining: 2m 55s
4050:	learn: 0.9596167	total: 13m 8s	remaining: 1m 27s
4499:	learn: 0.9611181	total: 14m 34s	remaining: 0us
0:	learn: 0.6149000	total: 175ms	remaining: 13m 7s
450:	learn: 0.9106292	total: 1m 28s	remaining: 13m 17s
900:	learn: 0.9302389	total: 2m 58s	remaining: 11m 54s
1350:	learn: 0.9396986	total: 4m 27s	remaining: 10m 22s
1800:	learn: 0.9459833	total: 5m 53s	remaining: 8m 50s
2250:	learn: 0.9500597	total: 7m 21s	remaining: 7m 21s
2700:	learn: 0.9533403	total: 8m 48s	remaining: 5m 51s
3150:	learn:

<a id="9"></a><h2></h2>
<div style="color:white;display:fill;border-radius:8px;
            background-color:#DAA520;font-size:150%;
            font-family:Nexa;letter-spacing:0.5px">
    <p style="padding: 8px;color:black;"><b>3.6 |  AdaBoostClassifier</b></p>
</div>

In [14]:
%%time
from sklearn.ensemble import AdaBoostClassifier
iterations = 100
ada_params = {
    'n_estimators': iterations,  
}


# Create Numpy arrays of train, test and target ( Survived) dataframes to feed into our models
y_train = train['target'].values
x_train = train[features].values # Creates an array of the train data
x_test = test[features].values # Creats an array of the test data
pred_test_list_clf= []
pred_train_list_clf = []
for seed in range(1):   # you can increase maximum value of range upto 10. I have choosen it to be 1 so that the runtime is shorter.
    print(f"{seed:2}")
    ada = SklearnHelper(clf=AdaBoostClassifier, seed=SEED, params=ada_params)
    ada_oof_train, ada_oof_test = get_oof(ada,x_train, y_train, x_test) 
    score = roc_auc_score(y_train, np.array(ada_oof_train))
    print(score)
    pred_train_list_clf.append(ada_oof_train)
    pred_test_list_clf.append(ada_oof_test) 
ada_oof_train=np.array(pred_train_list_clf).mean(axis=0)
ada_oof_test=np.array(pred_test_list_clf).mean(axis=0)
score = roc_auc_score(y_train, np.array(ada_oof_train))
print("final_AUC",score)

 0
0.7712009756956579
final_AUC 0.7712009756956579
CPU times: user 26min 46s, sys: 36.8 s, total: 27min 22s
Wall time: 27min 27s


<a id="10"></a><h2></h2>
<div style="color:white;display:fill;border-radius:8px;
            background-color:#DAA520;font-size:150%;
            font-family:Nexa;letter-spacing:0.5px">
    <p style="padding: 8px;color:black;"><b>3.7 |  RandomForestClassifier</b></p>
</div>

In [15]:
%%time
from sklearn.ensemble import RandomForestClassifier
rf_params = {
    'n_estimators': 100,
}

# Create Numpy arrays of train, test and target ( Survived) dataframes to feed into our models
y_train = train['target'].values
x_train = train[features].values # Creates an array of the train data
x_test = test[features].values # Creats an array of the test data
pred_test_list_clf= []
pred_train_list_clf = []
for seed in range(1): # you can increase maximum value of range upto 10. I have choosen it to be 1 so that the runtime is shorter.
    print(f"{seed:2}")
    rf = SklearnHelper(clf=RandomForestClassifier, seed=SEED, params=rf_params)
    rf_oof_train, rf_oof_test = get_oof(rf,x_train, y_train, x_test) 
    score = roc_auc_score(y_train, np.array(rf_oof_train))
    print(score)
    pred_train_list_clf.append(rf_oof_train)
    pred_test_list_clf.append(rf_oof_test)
rf_oof_train=np.array(pred_train_list_clf).mean(axis=0)
rf_oof_test=np.array(pred_test_list_clf).mean(axis=0)
score = roc_auc_score(y_train, np.array(rf_oof_train))
print("final_AUC",score)



 0
0.9097271466634649
final_AUC 0.9097271466634649
CPU times: user 36min 18s, sys: 3.76 s, total: 36min 22s
Wall time: 36min 30s


<a id="11"></a><h2></h2>
<div style="background-color:rgba(0, 153, 0, 0.5);border-radius: 15px 50px;display:fill">
    <h1><center>4. Meta Learning </center></h1>
</div>
Normalmente, um metamodelo é treinado nas previsões feitas por modelos de base em dados fora da amostra. Ou seja, os dados não usados para treinar os modelos de base são alimentados aos modelos de base, as previsões são feitas e essas previsões, juntamente com as saídas esperadas, fornecem os pares de entrada e saída do conjunto de dados de treinamento usado para ajustar o metamodelo. No entanto, neste caderno eu usei a previsão de todos os dados.

<br>
<div class="alert alert-block alert-info">


Anotações importantes:
1) A previsão da CNN, XGB e outras previsões são mescladas com alguns outros recursos dominantes e, em seguida, são usadas como dados de treinamento para o classificador de aprendizado Meta. <br>
2) Para evitar overfitting, um metamodelo deve ser ajustado com algoritmos leves. Aqui, usei regressão linear. Para fins de experimento, tentei vários outros algoritmos também.
.</div>
<div style="color:white;display:fill;border-radius:8px;
            background-color:#DAA520;font-size:150%;
            font-family:Nexa;letter-spacing:0.5px">
    <p style="padding: 8px;color:black;"><b>4.1 |  Preprocessing for Meta learning</b></p>
</div>


In [16]:
#merging all the Level-0 prediction of the traing data
Metainput_train2 =pd.DataFrame()
Metainput_train2['id']=train['id']
Metainput_train2['target']=train['target']
Metainput_train2['LGBM_train_pred'] = lgb_oof_train.ravel()
Metainput_train2['XGB_train_pred'] =  xgb_oof_train.ravel()
Metainput_train2['CNN_train_pred'] =  np.array(predtrain_list_cnn).mean(axis=0)
Metainput_train2['HIST_train_pred'] =  Hist_oof_train.ravel()
Metainput_train2['catboost'] =  cat_oof_train.ravel()
Metainput_train2['adaboost'] =  ada_oof_train.ravel()
Metainput_train2['rf'] =  rf_oof_train.ravel()
Metainput_train2.head(2)

,id,target,LGBM_train_pred,XGB_train_pred,CNN_train_pred,HIST_train_pred,catboost,adaboost,rf
0,0,0,0.0,0.0,0.000018,0.0,0.0,0.0,0.0
1,1,1,1.0,1.0,1.000000,1.0,1.0,1.0,1.0


In [17]:
#merging all the Level-0 prediction of the test data
Metainput_test2 =pd.DataFrame()
Metainput_test2['id']=test['id']
Metainput_test2['LGBM_train_pred'] = lgb_oof_test.ravel()
Metainput_test2['XGB_train_pred'] =  xgb_oof_test.ravel()
Metainput_test2['CNN_train_pred'] =  np.array(predtest_list_cnn).mean(axis=0)
Metainput_test2['HIST_train_pred'] =  Hist_oof_test.ravel()
Metainput_test2['catboost'] =  cat_oof_test.ravel()
Metainput_test2['adaboost'] =  ada_oof_test.ravel()
Metainput_test2['rf'] =  rf_oof_test.ravel()
Metainput_test2.head(2)


,id,LGBM_train_pred,XGB_train_pred,CNN_train_pred,HIST_train_pred,catboost,adaboost,rf
0,900000,1.0,1.0,0.999816,1.0,1.0,1.0,1.0
1,900001,1.0,1.0,0.999995,1.0,1.0,1.0,1.0


In [18]:
imp_features_notarget = [x for x in Metainput_train2.columns if x != 'target']
from sklearn.model_selection import StratifiedKFold

val = np.zeros(Metainput_train2.shape[0])
pred = np.zeros(Metainput_train2.shape[0])

x = Metainput_train2[imp_features_notarget].values
y = Metainput_train2['target'].values

<a id="12"></a><h2></h2>
<div style="color:white;display:fill;border-radius:8px;
            background-color:#DAA520;font-size:150%;
            font-family:Nexa;letter-spacing:0.5px">
    <p style="padding: 8px;color:black;"><b>4.1 |  Meta-learning classifier- LinearRegression</b></p>
</div>

it is extremly important that we do not over fit the Meta-learning classifier. For this reason very light weight classfier should be used. lets start with Linear regression first and see how it predicts.

In [19]:
from sklearn.linear_model import LinearRegression
model_lr = LinearRegression()
pred_list_lr = []
for seed in range(10):
    print(f"{seed:2}")
    folds = StratifiedKFold(n_splits=2, shuffle=True, random_state=seed)
    for fold_index, (train_index,val_index) in enumerate(folds.split(x,y)):
        print('Batch {} started...'.format(fold_index))
        bst = model_lr.fit(x[train_index],y[train_index])
        val[val_index] = model_lr.predict(x[val_index])
        print('auc of this val set is {}'.format(roc_auc_score(y[val_index],val[val_index])))
    pred_list_lr.append(model_lr.predict(Metainput_test2))


 0
Batch 0 started...
auc of this val set is 0.9985263181306686
Batch 1 started...
auc of this val set is 0.9985022477726827
 1
Batch 0 started...
auc of this val set is 0.9985011985081995
Batch 1 started...
auc of this val set is 0.9985348363293567
 2
Batch 0 started...
auc of this val set is 0.9985016904823726
Batch 1 started...
auc of this val set is 0.9985212664996728
 3
Batch 0 started...
auc of this val set is 0.998480301624512
Batch 1 started...
auc of this val set is 0.9985384342750969
 4
Batch 0 started...
auc of this val set is 0.9985106971204466
Batch 1 started...
auc of this val set is 0.9985086921227948
 5
Batch 0 started...
auc of this val set is 0.9985463529525727
Batch 1 started...
auc of this val set is 0.9984842139426428
 6
Batch 0 started...
auc of this val set is 0.9985282381306455
Batch 1 started...
auc of this val set is 0.9984901687105703
 7
Batch 0 started...
auc of this val set is 0.9984741102181061
Batch 1 started...
auc of this val set is 0.9985588492809099
 

### Based on the above its can be seen that Linear repression based Meta-classifier is unable to increase the accuracy  when compared with level-0 CNN model.
### I have tried various models as Meta-classifier(Catboost, XGB, RF and Ridge...) with an object not to overfit the model. The following has proven to be best one.

<a id="13"></a><h2></h2>
<div style="color:white;display:fill;border-radius:8px;
            background-color:#DAA520;font-size:150%;
            font-family:Nexa;letter-spacing:0.5px">
    <p style="padding: 8px;color:black;"><b>4.2 |  Meta learning classifier- RandomForestRegressor</b></p>
</div>

In [20]:
from sklearn.metrics import auc
clf = RandomForestClassifier(
    n_estimators=5,
)
def compute_roc_auc(index):
    y_predict = clf.predict_proba(x.iloc[index])[:,1]
    fpr, tpr, thresholds = roc_curve(y.iloc[index], y_predict)
    auc_score = auc(fpr, tpr)
    
    return fpr, tpr, auc_score
cv = StratifiedKFold(n_splits=5, random_state=123, shuffle=True)
results = pd.DataFrame(columns=['training_score', 'test_score'])
fprs, tprs, scores = [], [], []


x = Metainput_train2[imp_features_notarget]
y = Metainput_train2['target']
x_test = Metainput_test2[imp_features_notarget].values
pred_metarf_list = []
for (train_meta, test_meta), i in zip(cv.split(x, y), range(10)):
    clf.fit(x.iloc[train_meta], y.iloc[train_meta])
    _, _, auc_score_train = compute_roc_auc(train_meta)
    print(auc_score_train)
    fpr, tpr, auc_score = compute_roc_auc(test_meta)
    scores.append((auc_score_train, auc_score))
    fprs.append(fpr)
    tprs.append(tpr)
    clf.fit(x.iloc[train_meta], y.iloc[train_meta])
    pred_metarf_list.append( (clf.predict_proba(Metainput_test2[imp_features_notarget].values)[:,1]))

0.9998820698557146
0.9998708659106026
0.9998845790548639
0.9998731674951156
0.9998663221599727


<img src= "https://coolwallpapers.me/th700/6044454-homer-simpson-wowooooooooo-homer-simpson-simpsons-the-simpsons.jpg" alt ="TPA MAY" style='width: 400px;'>
<br>

## The meta-classfier is predicting the training set with a score of 0.9998 for all the folds, which is great. The LB score of test data is 0.99545. But I belive, similar to TPS March2022 and other previous competitions Public LB score is bit of misleading. 

In [21]:
submission = test[['id']].copy()
submission['target'] = np.array(pred_metarf_list).mean(axis=0)
submission.to_csv('submission.csv', index=False)
submission

,id,target
0,900000,9.600000e-01
1,900001,9.600000e-01
2,900002,0.000000e+00
3,900003,6.865146e-07
4,900004,9.600000e-01
...,...,...
699995,1599995,3.600000e-01
699996,1599996,9.600000e-01
699997,1599997,0.000000e+00
699998,1599998,6.865146e-07


<a id="14"></a><h2></h2>
<div style="background-color:rgba(0, 153, 0, 0.5);border-radius: 15px 50px;display:fill">
    <h1><center>4. References </center></h1>
</div>

https://www.kaggle.com/code/javigallego/tps-apr22-eda-fe-lstm-tutorial#1-|-Introduction <br>
https://www.kaggle.com/code/andreshg/tps-apr-data-visualization-and-highlights <br>
https://machinelearningmastery.com/histogram-based-gradient-boosting-ensembles/ <br>
https://www.kaggle.com/code/serigne/stacked-regressions-top-4-on-leaderboard <br>
https://yanpuli.github.io/posts/2018/01/blog-post-10/ <br>